In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet_f\\validation'

In [3]:
data_f=pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f=pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

In [4]:
data_f.head()

,user_id,sig_id,fakeness
0,352,1,True
1,352,2,True
2,352,3,True
3,352,4,True
4,352,5,True


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.492046,0.000000,0.262247,0.000000,0.000000,0.0,0.887893,0.0,0.0,0.000000,...,1.213698,0.000000,0.000000,0.350542,0.000000,0.022946,0.685446,0.515102,0.000000,2.431604
1,0.462773,0.158136,0.000000,0.000000,0.354178,0.0,0.154205,0.0,0.0,0.000000,...,0.815964,0.000000,0.000000,0.229273,0.145647,0.139663,0.616705,0.079669,0.000000,1.367158
2,0.000000,0.214853,0.000000,0.210621,0.000000,0.0,0.667727,0.0,0.0,0.754772,...,0.878875,0.000000,0.000000,0.086916,0.671329,0.000000,0.993817,1.146683,0.000000,0.858650
3,0.000000,0.263884,0.000000,0.000000,0.964106,0.0,0.360634,0.0,0.0,0.000000,...,0.514347,0.171412,0.057328,1.145879,0.000000,0.000000,0.062078,0.152515,0.000000,0.573888
4,0.000000,0.608160,0.000000,0.000000,0.588335,0.0,0.000000,0.0,0.0,0.000000,...,0.649497,0.607849,0.051503,0.731876,0.000000,0.000000,0.456680,0.713516,0.237488,0.063648


In [6]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,23))] # TODO is actually 10, fix

In [13]:
scores_sum = 0
run_count = 0
gens_sum = 0
gens_r_count = 0
fakes_sum = 0
fakes_r_count = 0
randoms_sum = 0
randoms_r_count = 0
DECISION_THRESHOLD = 0.75
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    y_train = data_f_gen_12['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    class_weight = (data_f_gen_12.shape[0]-data_f_gen_12[y_train].shape[0])/data_f_gen_12[y_train].shape[0]
    clf_2 = MLPClassifier(hidden_layer_sizes=(150,100,50),random_state=1923)
    clf = SVC(class_weight='balanced',gamma='scale',probability=True,random_state=1923, kernel='linear') # gens avg: 0.8593333333333333 fakes_avg: 0.9433333333333331 randoms_avg: 1.0 cumulative_avg: 0.9342222222222221
    clf.fit(X_train, y_train)
    clf_2.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[(data_f_fake['user_id']==user_id) & (data_f_fake['sig_id'].isin(np.arange(1,11)))]
    y_valid_f = len(y_valid_fakes.index)*[False]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    score = accuracy_score(y_valid_f, ((clf.predict_proba(X_valid_f)/2)+(clf_2.predict_proba(X_valid_f)/2))[:,1]>= DECISION_THRESHOLD)
    fakes_sum += score
    fakes_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid_g = len(y_valid_gens.index)*[True]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    score = accuracy_score(y_valid_g, ((clf.predict_proba(X_valid_g)/2)+(clf_2.predict_proba(X_valid_g)/2))[:,1]>= DECISION_THRESHOLD)
    gens_sum += score
    gens_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid_rf = len(y_valid_rand_fakes.index)*[False]
    X_valid_rf = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid_rf,y_valid_rf)
    randoms_sum += score
    randoms_r_count += 1 
    print ("gens avg:", float(gens_sum)/gens_r_count, "fakes_avg:", float(fakes_sum)/fakes_r_count, 
            "cumulative_avg:", (float(gens_sum)/gens_r_count)/2 + (float(fakes_sum)/fakes_r_count)/2)


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

gens avg: 1.0 fakes_avg: 1.0 cumulative_avg: 1.0


  2%|#6                                                                                 | 1/50 [00:01<00:59,  1.22s/it]

gens avg: 1.0 fakes_avg: 0.95 cumulative_avg: 0.975


  4%|###3                                                                               | 2/50 [00:02<00:56,  1.18s/it]

...

 96%|##############################################################################7   | 48/50 [01:01<00:02,  1.24s/it]

gens avg: 0.897959183673469 fakes_avg: 0.9061224489795917 cumulative_avg: 0.9020408163265303


 98%|################################################################################3 | 49/50 [01:02<00:01,  1.19s/it]

gens avg: 0.8999999999999997 fakes_avg: 0.9039999999999998 cumulative_avg: 0.9019999999999997


100%|##################################################################################| 50/50 [01:03<00:00,  1.19s/it]


In [8]:
clf.predict(X_valid_f)

array([ True,  True,  True, False, False, False, False, False, False,
       False])